# Smart Clinic Image Loader
### By Patrick Myers and Brian Morris

This code is in progress. It will take a voice command from the SmartClinic Alexa skill to open some piece of information. Right now, the only functions written are to open an image file with a specific date or by a relative date in a specific folder. Future functions will include:

**More customization for opening files:**
* Open lab results (first by specific date)
* Open all files by name
* Open all files by relative info such as 'most recent'
* Open files when path is not certain

**More features for the SmartClinic:**
* Automatically add the opened files to the patient note
* Record the diagnosis and the opened files to begin learning patterns

Load in the libraries

In [1]:
import spacy                        # NLP toolbox of choice
import pandas as pd                 # A useful data storage toolbox
from datetime import datetime       # Python formats for dates to match files
from pathlib import Path            # Manipulation of paths to find files
from os.path import isfile, join    # Simple toolbox for only looking at files
import os                           # All operating system commands
import calendar                     # Necessary for conversion to Month strings
import sys                          # Accessing the system directly
import subprocess                   # Calling subprocesses to open files
from PIL import Image               # A more convenient way to open images                                                                               

Load in the NLP dictionary. Must be downloaded first. Since this dictionary is not trained for medical jargon, some commands may struggle to be interpreted

In [2]:
nlp = spacy.load("en_core_web_sm")

The method for calling a subprocess to open an image. Should be platform independent, but is messy. Not currently used.

In [86]:
def openImage(path):
    imageViewerFromCommandLine = {'linux':'xdg-open',
                                  'win32':'explorer',
                                  'darwin':'open'}[sys.platform]
    subprocess.run([imageViewerFromCommandLine, path])

This function will parse through the methods by which the user wants to access the image, whether that be by its date, name, etc. Calls the appropriate function for that task. Currently, only by date by relative position work

In [3]:
def get_img(command, when):
    if command == 'ByDate':
        get_img_bydate(when)
    elif command == 'ByRelative':
        get_img_byrel(when)
    

This is the function to open an image given the date of the image.  It is likely that the image will not actually be put into the filesystem on the day that the image was taken. If this is the case, we will need to add some functionality to find the file close to the requested date. 

The input date is in Python's datetime format. The checked images have their creation time pulled, but this time is in Python's time format. Some work is done to convert this time to the same datetime as the input date for comparison.

The easier image opening method is used here, using PILLOW

In [4]:
def get_img_bydate(dates):
    mypath = str(Path().absolute())
    imagepath = join(mypath,'Scans')
    image_files = [f for f in os.listdir(imagepath) if os.path.isfile(join(imagepath,f))]
    name_date = []
    for f in image_files:
        date = os.path.getctime(join(imagepath,f))
        date = datetime.fromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')
        date = date.split(" ")
        date = date[0]
        date = (date.split("-"))
        month = int(date[1])
        month = calendar.month_name[month]
        month = month[0:3]
        date[1] = month
        date = [date[1], date[2], date[0]]
        date = " ".join(date)
        date = datetime.strptime(date, '%b %d %Y')
        name_date.append((f, date))
    for i in range(0,len(name_date)):
        if name_date[i][1] == dates[0]:
            imgpath = join(imagepath, name_date[i][0])
            img = Image.open(imgpath)
            img.show()
            

This is the function to open an image by its relative position. Currently, only end images (the least or most recent) are accessible this way. 

The function finds all the image files available, then their corresponding date. Putting the two data points in a tuple allows us to sort the images by date. The most (or least) recent image is then opened.

All specific functions are explained in get_img_bydate

In [5]:
def get_img_byrel(relPos):
    mypath = str(Path().absolute())
    imagepath = join(mypath,'Scans')
    image_files = [f for f in os.listdir(imagepath) if os.path.isfile(join(imagepath,f))]
    name_date = []
    for f in image_files:
        date = os.path.getctime(join(imagepath,f))
        date = datetime.fromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')
        date = date.split(" ")
        date = date[0]
        date = (date.split("-"))
        month = int(date[1])
        month = calendar.month_name[month]
        month = month[0:3]
        date[1] = month
        date = [date[1], date[2], date[0]]
        date = " ".join(date)
        date = datetime.strptime(date, '%b %d %Y')
        name_date.append((f, date))
    name_date = sorted(name_date, key=lambda x: x[1])
    if relPos == 'LR':
        pos = 0
    elif relPos == 'MR':
        pos = len(name_date)-1
    imgpath = join(imagepath, name_date[pos][0])
    img = Image.open(imgpath)
    img.show()

This first test simulates a voice command for opening an image by its date. The function merges phrases (so February 11 2019 is seen as one word instead of three), then finds the dates within the request. The date is converted to a Python datetime for comparison.

The test then makes sure that the direct object has the correct action word attached to it and is one of the files that we can provide. Currently, this is done by searching for the direct object, then looping back to the acting verb. This verb is checked against a list of terms that could be used to request a file. Then the direct object itself is checked to see if it is in the acceptible image or lab terms, directing the function call. This method of checking against common phrases is likely naive, so more research will be done and improvements will be made.

If the request makes sense, then the image with the correct date is opened.

In [6]:
def Test1():
    doc = nlp(u'Open the image from February 11 2019')
    actions = ['open', 'show']
    imageterms = ['image', 'scan', 'the image', 'the scan']
    labterms = ['lab', 'test', 'the lab', 'the test']
    spans = list(doc.ents) + list(doc.noun_chunks)
    for span in spans:
        span.merge()
    dates = []
    for date in filter(lambda w: w.ent_type_ == "DATE", doc):
        date_text = date.text
        mdy = date_text.split(' ')
        mdy[0] = mdy[0][0:3]
        date_text = " ".join(mdy)
        datetime_object = datetime.strptime(date_text, '%b %d %Y')
        dates.append(datetime_object)
    image = False
    lab = False
    for sent in doc.sents:
        for i, word in enumerate(sent):
            if word.head is word:
                head_idx = 0
            else:
                head_idx = doc[i].head.i+1
            if word.dep_ == 'dobj':
                if doc[i].head.lemma_ in actions:
                    if word.text in imageterms:
                        print('Opening the image')
                        image=True
                    elif word.text in labterms:
                        print('Opening the lab')
                        lab=True
            #print("%d\t%s\t%s\t%s\t%s\t%s\t%d\t%s\t%s\t%s"%(
            #    i+1, # There's a word.i attr that's position in *doc*
            #    word.text,
            #    '_',
            #    word.pos_, # Coarse-grained tag
            #    word.tag_, # Fine-grained tag
            #    '_',
            #    head_idx,
            #    word.dep_, # Relation
            #    '_', '_'))
    if dates and image:
        get_img('ByDate', dates)

The second test simulates a voice command to open a file based on some relative information. Right now, the only relative term tested is 'recent', but this will be updated when more time is spent in clinic. The test searches for the adjective modifier (in this case, the term 'recent') and notes its index. It then searches through all of the words that point to the term 'recent' to see if it is negated by 'least'. 

Based on this negation or lack thereof, the test calls to the get_img function with either the indicator of 'MR' for most recent or 'LR' for lease recent

In [9]:
def Test2(): 
    doc = nlp(u'Open the most recent image')
    actions = ['open', 'show']
    imageterms = ['image', 'scan', 'the image', 'the scan']
    labterms = ['lab', 'test', 'the lab', 'the test']
    relind = -1
    MRbool = False
    LRbool = False
    for sent in doc.sents:
        for i, word in enumerate(sent):
            if word.dep_ == 'amod':
                relind = i
    for sent in doc.sents:
        for i, word in enumerate(sent):
            if word.head.i == relind:
                if word.text == 'most' and doc[relind].text == 'recent':
                    MRbool = True
                elif word.text == 'least' and doc[relind].text == 'recent':
                    LRbool = True
    if MRbool:
        print('Opening the most recent image')
        get_img('ByRelative', 'MR')

Run all the tests

In [10]:
Test1()
Test2()

Opening the image
Opening the most recent image
